# Analyzign Models Best Suited for Gaps Project

## Imports

In [1]:
import gc
import re
import csv
import json
import torch
import wandb
import random

import pandas as pd
# import numpy as np

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer, DPRReader, Trainer, TrainingArguments

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None) 
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_seq_item', None)

## Data Preprocessing

In [2]:
with open ('data/all-csu-codes.csv', 'r') as c_data:
    csv_reader = csv.reader(c_data) 
    courses_data = list(csv_reader)
    
courses_df = pd.DataFrame(columns=['Courses', 'Skills'])

for idx in range(0, len(courses_data)):
    skill_list = courses_data[idx][1:-1]
    skill_list = [skill.title() for skill in skill_list]
    skill_list = [re.sub(r'\b(vs|Vs)\b', 'VS', skill) for skill in skill_list]

    row = pd.DataFrame({'Courses': courses_data[idx][0], 'Skills':[skill_list]})
    courses_df = pd.concat([courses_df, row], ignore_index=True)

courses_df.to_csv('data/dpr_courses_data.csv', index=False)

In [3]:
with open('data/descriptions.txt', 'r') as j_data:
    csv_reader = csv.reader(j_data, delimiter='|')
    jobs_data = list(csv_reader)

jobs_df = pd.DataFrame(columns=['Job_Title', 'Job_Description', 'Required_Skills'])
                    
for row in jobs_data:
    if len(row) == 3:
        job_title = row[0].strip().strip('"') 

        job_description = row[1].strip().strip('"')
        job_description = re.sub(r'\bDESCRIPTION\b', '', job_description)
    
        skills = row[2].strip().strip('"')
        skill_list = [skill.strip().strip('"') for skill in skills.split(',')]
        cap_skill_list = [skill.title() for skill in skill_list]
        cleaned_skills = [re.sub(r'\s?\(.*?\)', '', skill) for skill in cap_skill_list]

        row = pd.DataFrame({'Job_Title': job_title, 'Job_Description': job_description, 'Required_Skills': [skill_list]})
        jobs_df = pd.concat([jobs_df, row], ignore_index=True)
            
jobs_df.to_csv('data/dpr_jobs.csv', index=False)

In [4]:
check_course = next(item for item in courses_data if item[0] == 'CS470')
print(check_course)
print('********************************************')
print(courses_df.head(1))

['CS470', 'Assembly', 'C Language', 'Digital Gates', 'Instruction Set Architectures', 'Number Representation']
********************************************
  Courses  \
0   CS462   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Skills  
0  [3D Modeling, Animations, Assigning 3D Object Properties, Blender, Calculating Lights, Shades,

In [5]:
courses_df

,Courses,Skills
0,CS462,"[3D Modeling, Animations, Assigning 3D Object Properties, Blender, Calculating Lights, Shades, And, Camera Rendering, Character Animation, Character Design, Collisions, Constructing 3D Scenes, Design 3D, Develop 3D, Event Triggering, Finite State Machine, Fourier Transform, Game Design, Geometric Image Manipulation, Human Aware Ai, Immersive 3D Worlds, Implementing Animation To Chara, Implementing Movement To Charac, Manipulate Lighting, Manipulate Rendering, Manipulating 3D Object Properti, Movement, Principles Of Lighting, Principles Of Rendering, Real Time Rendering Pipeline, Scene Composition, Scene Layout, Scripting Interactive Behaviors, Scripting Interactive Elements, Tangent Space, Unity, Vectors]"
1,CS314,"[Agile, Black Box Testing, Burndown Charts, Clean Code, Cmmi, Code Climate, Code Quality, Communication, Compatability Standards, Concurrency, Configuration Management, Continuous Integration, Databases, Development Environments, Devops, Docker, Docker Container, Establishing Interpersonal Rela, Git, Github, Github Actions, Github Projects, Github Repository, Individual Metrics, Integration Tests, Intellij, Java, Java Concurrency, Java Spark, Json, Junit, Kml, Linux, Mariadb, Maven, Networking, Npm, Optimization, Peer Evaluation, Peer Review, Port Forwarding, Postman, Problem Solving, Product Integration, Project Management, Project Planning, Refactoring, Remote Development, Rest Api, Retrospectives, Scrum, Slack, Slf4J, Software Development Practices, Source Control, Sprint Planning, Sql, Story Boards, Story Sizing, Task Breakdown, Team Development Experience, Team Diversity, Team Metrics, Teamwork, Test Driven Development, Tuckman'S Model, Unix, Use Case Testing, Verification, VS Code, Webpack, White Box Testing]"
2,CS165,"[Algorithms, Assertions, B+ Trees, Binary Search Trees, Black Box Testing, Branching Recursion, Data Structures, Dequeues, Expression Trees, Generics, Graph, Hashmap, Infix, Inheritance, Java, Linkedlists, Object Oriented Principles, Object Oriented Programming, Pcre, Polymorphism, Postfix, Prefix, Priority Queues, Problem Solving, Queues, Regex, Stack, Unit Testing]"
3,CS201,"[Argument Construction, Computer Solution Designing, Computer Solution Implementatio, Computer Solution Operations, Decision Making, Ethical Dilemma Analysis, Ethical Dilemma Problem Solving, Ethics Analysis, Legal Obligations, Moral Obligations, Philosophy, Professional Code Of Ethics]"
4,CS370,"[Commercial Operating Systems, Containers, Deadlocks, Deadlocks Management, Design Threaded Programs, File System Architecture, Interprocess Communication, Kernel Threads, Memory Management, Open Source Operating Systems, Operating Systems, Process Synchronization, Processes Management, Resource Management, Scheduling Algorithms, Storage Architecture, Symmetric Multiprocessing, Synchronization, Task Synchronization, Thread Management, Threads, Type-1 Hypervisors, Type-2 Hypervisors, User Threads]"
5,CS150,"[Conditionals, Data Analysis, Data Visualization, Dictionaries, Functions, Hcc, Html, I O Console, I O File, Libraries, Lists, Loops, Operator, Privacy, Python, Research, Research Design, Security, Sets]"
6,CS110,"[Data Analysis, Data Manipulation, Data Visualization, Hardware, Privacy, Research, Security]"
7,CS455,"[Abstraction, Algorithms Underpinning P2P Sys, Amazon Dynamo, Architectural Styles For Distri, Berkley Algorithm, Bittorrent, Brewer'S Cap Theorem, Build Distributed Systems, Build Scalable Servers, Casual Consistency, Central-Server Algorithm, Centralized Single Cpu Systems, Chord, Clock Synchronization In Distri, Combiner Functions, Compound Actions, Concurrency Primitives, Concurrent Collections, Concurrent Programming, Conditions Requirements For Dis, Confinement, Consensus, Consistency, Consistency Protocol, Consistent Ordering Of Operatio, Core Architecture Framework, Cristian'S Algorithm, Data And Client Centric Consist, Data Synchronization, Design And Build Distrubuted Fa, Design Clo

In [6]:
check_job = [item for item in jobs_data if item[0] == 'Yahoo_Software_Dev_Engineer']
print(check_job)
print('********************************************')
print(jobs_df.head(1))

[['Yahoo_Software_Dev_Engineer', "Who We're Looking For- Junior Software Engineer We hire engineers who love the web, but can see its cracks and its future, too. We look for people who are exceptionally who are exceptionally imaginative, collaborative, and truly excited about tech. Our DSP Reporting team is currently looking for talented full-stack engineers to design, implement, and support robust, scalable, and high-quality reporting solutions Your Responsibilities - Develop and enhance a state-of-the-art reporting and analytics platform. - Build intuitive front-end UIs for reporting and analytics using React. - Develop microservices to power reporting and analytics solutions. - Write clean, maintainable, and performant code, including unit tests and refactoring when needed. - Collaborate with designers and developers to define and deliver new features. - Participate in system architecture reviews, code reviews, performance tuning, and production support. - Own and deliver projects r

In [7]:
jobs_df

Job_Title  \
0                              Adobe_AI_ML_Engineer   
1                                  Adobe_Junior_SDE   
2                 Adobe_Software_Engineering_Intern   
3                   Adobe_Software_Quality_Engineer   
4                Adobe_Software_Solutions_Architect   
5                 Amazon_Cloud_Support_Engineer_ETL   
6                 Amazon_Graduate_Software_Engineer   
7                                  Amazon_SDE_AI_ML   
8                                  Amazon_SDE_Cloud   
9                           Amazon_SDE_Embedded_Dev   
10                                     Amazon_SDE_I   
11                    Amazon_SDE_Query_Optimization   
12                            Amex_Backend_Engineer   
13                               Amex_Engineer_Java   
14                           Amex_Software_Engineer   
15                  Apple_Software_Engineer_WatchOS   
16                         Apple_SWE_Internal_Tools   
17                            Apple_Webkit_Engineer   
18                   Comcast_Software_Engineer_Coop   
19                Figma_Data_Scientist_Early_Career   
20                              Figma_Web_Developer   
21                         Github_Software_Engineer   
22                       Github_Software_Engineer_2   
23                                Google_ML_Analyst   
24                       Google_ML_Engineer_Silicon   
25          Google_Software_Engineer_Youtube_Design   
26                               Google_UX_Designer   
27                               HPE_AI_ML_Engineer   
28                              HPE_Cloud_Developer   
29                               HPE_Data_Scientist   
30                 HPE_Electrical_Hardware_Engineer   
31                                     HPE_Engineer   
32                  HPE_Engineering_Program_Manager   
33                   HPE_Experience_Design_Engineer   
34                          HPE_Industrial_Engineer   
35                          HPE_Mechanical_Engineer   
36                             HPE_Product_Engineer   
37                              HPE_Product_Manager   
38                   HPE_Quality_Assurance_Engineer   
39                             HPE_Quality_Engineer   
40                            HPE_Research_Engineer   
41                            HPE_Software_Designer   
42                         HPE_SW_Engineer_Firmware   
43                    HPE_Systems_Software_Engineer   
44                                HPE_Test_Engineer   
45                                HPE_VLSI_Engineer   
46                  HP_Associate_Software_Developer   
47                     HP_Software_Quality_Engineer   
48                           Meta_Software_Engineer   
49                   Meta_Software_Engineer_Android   
50                  Meta_Software_Engineer_Codesign   
51           Meta_Software_Engineer_Computer_Vision   
52                  Meta_Software_Engineer_Frontend   
53            Meta_Software_Engineer_Infrastructure   
54                      Microsoft_Applied_Scientist   
55                              Microsoft_Fullstack   
56                    Microsoft_Security_Researcher   
57          Microsoft_Software_Engineering_Security   
58                Microsoft_Software_Engineer_AI_ML   
59                 Netflix_Software_Engineer_Intern   
60  OpenAI_Software_Engineering_Applied_Engineering   
61               OpenAI_Software_Engineer_Real_Time   
62                          Paypal_Android_Engineer   
63                             Paypal_Fresh_Grad_SE   
64                     Paypal_Platforms_Engineering   
65                         Paypal_SE_Growths_Market   
66                         Paypal_Software_Engineer   
67                        Sony_Blockchain_Developer   
68                    Sony_Software_Design_Engineer   
69              Spotify_Backend_Engineer_Audiobooks   
70           Spotify_Backend_Engineer_Data_Platform   
71                Spotify_Data_Scientist_Audiobooks   
72                  Spotify

In [8]:
del check_course, check_job, j_data, jobs_data, csv_reader, c_data, courses_data
gc.collect()
courses_df.shape, jobs_df.shape

((35, 2), (79, 3))

### Collect All Skills from CSU Course Data

In [ ]:
def get_all_acquired_skills(courses_df):
    all_acquired_skills = set() 
    
    for skills in courses_df['Skills']:
        all_acquired_skills.update(skills)

    return all_acquired_skills

In [10]:
all_acquired_skills = get_all_acquired_skills(courses_df)
list(all_acquired_skills)[0:9]

['Moral Obligations',
 'Code Styles',
 'Prefetching Instructions',
 'Smart Contracts',
 'Cyber Threat Intelligence',
 'Computer Output',
 'Debugging',
 'C',
 'Intrusion Detection']

### Create Corpus for CSU Courses 

In [11]:
def create_corpus(courses_df):
    corpus = []
    for idx, row in courses_df.iterrows():
        document = {
            "id": f"csu_course_{idx}",
            "title": row['Courses'],
            "text": f"Skills: {', '.join(row['Skills'])}",
            "metadata": {"course_name": row['Courses'], "skills_acquired": row['Skills']}
        }
        corpus.append(document)
    return corpus


courses_corpus = create_corpus(courses_df)

with open("data/dpr_course_corpus.json", "w") as f:
    json.dump(courses_corpus, f, indent=4)

json.dumps(courses_corpus[0], indent=4)

'{\n    "id": "csu_course_0",\n    "title": "CS462",\n    "text": "Skills: 3D Modeling, Animations, Assigning 3D Object Properties, Blender, Calculating Lights, Shades, And, Camera Rendering, Character Animation, Character Design, Collisions, Constructing 3D Scenes, Design 3D, Develop 3D, Event Triggering, Finite State Machine, Fourier Transform, Game Design, Geometric Image Manipulation, Human Aware Ai, Immersive 3D Worlds, Implementing Animation To Chara, Implementing Movement To Charac, Manipulate Lighting, Manipulate Rendering, Manipulating 3D Object Properti, Movement, Principles Of Lighting, Principles Of Rendering, Real Time Rendering Pipeline, Scene Composition, Scene Layout, Scripting Interactive Behaviors, Scripting Interactive Elements, Tangent Space, Unity, Vectors",\n    "metadata": {\n        "course_name": "CS462",\n        "skills_acquired": [\n            "3D Modeling",\n            "Animations",\n            "Assigning 3D Object Properties",\n            "Blender",\n 

### Create the Training Dataset

In [ ]:

def all_class_comparison(jobs_df, courses_df, all_acquired_skills):
    training_data = []
    context = "\n".join([f"Course: {course} -- Skills: {', '.join(skills)}"
                        for course, skills in zip(courses_df["Courses"], courses_df["Skills"])])

    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job["Required_Skills"])

        missing_skills = job_skills - all_acquired_skills

        training_data.append({
            'question': job_text,
            'context': context,
            'answer': list(missing_skills)
        })

    return training_data


def compare_individual_course(jobs_df, courses_df):
    training_data = []
     
    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job['Required_Skills'])  
        
        for _, course in courses_df.iterrows():
            course_name = course['Courses']
            course_skills = set(course['Skills'])  
            
            missing_skills = job_skills - course_skills
            context = f"Course: {course_name} -- Skills: {', '.join(course_skills)}"

            if missing_skills == job_skills:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': list(job_skills)
                })
            elif len(missing_skills) == 0:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': []
                })
            else:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': list(missing_skills)
                })

    return training_data


def create_schedule_data(jobs_df, schedules):
    training_data = []

    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job["Required_Skills"])

        for sched in schedules:
            context = "\n".join([f"Course: {course} -- Skills: {', '.join(skills)}"
                        for course, skills in zip(sched["Courses"], sched["Skills"])])
            sched_skills = [item for sublist in sched["Skills"].tolist() for item in sublist]
            missing_skills = job_skills - set(sched_skills)

            if missing_skills == job_skills:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': list(job_skills)
                })
            elif len(missing_skills) == 0:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': []
                })
            else:
                training_data.append({
                    'query': job_text,
                    'context': context,
                    'answer': list(missing_skills)
                })

    return training_data


def get_courseloads(jobs_df, courses_df, number_of_schedules=20):
    core_classes = ['CS150', 'CS164', 'CS152', 'CS162', 'CS201', 'CS165', 'CS220', 
                    'CS270', 'CS250', 'CS314', 'CS370', 'CS320', 'CS214']
    
    elective_courses_df = courses_df[~courses_df['Courses'].isin(core_classes)]

    schedules_df = []
    used_schedules = set()

    while len(schedules_df) < number_of_schedules:
        l_4_courses = elective_courses_df[elective_courses_df['Courses'].str.startswith('CS4')]
        l_3_4_courses = elective_courses_df[elective_courses_df['Courses'].str.startswith('CS3') | elective_courses_df['Courses'].str.startswith('CS4')]
        other_courses = elective_courses_df[~elective_courses_df['Courses'].str.startswith('CS3') & ~elective_courses_df['Courses'].str.startswith('CS4')]

        l_4_sample = random.sample(l_4_courses['Courses'].tolist(), 2)

        l_3_4_filtered = l_3_4_courses[~l_3_4_courses['Courses'].isin(l_4_sample)]
        l_3_4_sample = random.sample(l_3_4_filtered['Courses'].tolist(), 2)

        all_sampled_courses = l_4_sample + l_3_4_sample
        other_courses_filtered = other_courses[~other_courses['Courses'].isin(all_sampled_courses)]
        other_sample = random.sample(other_courses_filtered['Courses'].tolist(), 1)

        sched_courses = core_classes + l_4_sample + l_3_4_sample + other_sample
        sched_df = courses_df[courses_df['Courses'].isin(sched_courses)].copy()

        sched_tuple = tuple(sorted(sched_df['Courses'].tolist()))
        if sched_tuple not in used_schedules:
            schedules_df.append(sched_df)
            used_schedules.add(sched_tuple)

    training_data = create_schedule_data(jobs_df, schedules_df)

    return training_data


def create_training_data(jobs_df, courses_df, all_acquired_skills, num_negatives=4):
    training_data = all_class_comparison(jobs_df, courses_df, all_acquired_skills)
    print(f"All: {training_data[-1]}\n")

    training_data = training_data + compare_individual_course(jobs_df, courses_df)
    print(f"Individual: {training_data[-1]}\n")
    
    training_data = training_data + get_courseloads(jobs_df, courses_df)
    print(f"Course load: {training_data[-1]}\n")

    return training_data

# training_data, all_job_skills = create_training_data(jobs_df, courses_df, all_acquired_skills)

training_data = create_training_data(jobs_df, courses_df, all_acquired_skills)


# check = set(all_job_skills[0]) - set(training_data[0]["answer"])
# check2 = set(training_data[0]["answer"]) - set(all_job_skills[0])
# print(check) 
# print(check2)

(2765, 3)

In [ ]:
td_df = pd.DataFrame(training_data, columns=['question', 'context', 'answer'])
print(td_df.shape)

td_df.to_csv('data/dpr_training_data.csv', index=False)

del td_df
gc.collect()

(1694, 3)


66

## Question (Jobs) and Context (Courses) Encoding for DPR Training

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-multiset-base')
# context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-multiset-base')

# question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-multiset-base')
# context_tokenizer = DPRContextEncodeTokenizer.from_pretrained('facebook/dpr-ctx_encoder-multiset-base')

# train_data = {
#     'question': [entry['question'] for entry in dpr_training_data],
#     'context': [entry['context'] for entry in dpr_training_data],
#     'answer': [entry['answer'] for entry in dpr_training_data]
# }

# train_dataset = Dataset.from_dict(train_data)

# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     logging_dir='./logs',
#     logging_steps=100,
#     evaluation_strategy="steps",
#     save_steps=1000,
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset
# )

# trainer.train()

## Create and Initialize Dataloader Class

In [ ]:
# def collate_fn(batch):
#     question_input_ids = [item['question_input_ids'] for item in batch]
#     context_input_ids = [item['context_input_ids'] for item in batch]
    
#     question_input_ids_padded = pad_sequence(question_input_ids, batch_first=True, padding_value=0)  # Padding value 0 (or any value as required)
#     context_input_ids_padded = pad_sequence(context_input_ids, batch_first=True, padding_value=0)
    
#     return {
#         'question_input_ids': question_input_ids_padded,
#         'context_input_ids': context_input_ids_padded,
#     }

# class DPRDataset(Dataset):
#     def __init__(self, data):
#         self.data = data  # List of (question, context) pairs
#         self.tokenizer = tokenizer  # Your tokenizer
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         question, context = self.data[idx]
#         question_encoding = self.tokenizer(question, truncation=True, padding='max_length', return_tensors="pt")
#         context_encoding = self.tokenizer(context, truncation=True, padding='max_length', return_tensors="pt")

#         return {
#             'question_input_ids': question_encoding['input_ids'].squeeze(0),
#             'context_input_ids': context_encoding['input_ids'].squeeze(0),
#         }

# train_dataloader = DataLoader(training_data, batch_size=32, collate_fn=collate_fn)

## WandB Evaluation Setup

In [ ]:
# key_file = r'C:\Development\cs580B3\term_project_model\WANDB_API_KEY.txt' 

# with open(key_file, "r") as f:
#     api_key = f.read().strip()

# # Log into WandB with the API key
# wandb.login(key=api_key)

## Set-Up HuggingFace Evaluations

In [ ]:
# learning_rate = 1e-5
# batch_size = 4
# epochs = 3
# neg_sample_size = 4
# warmup_steps = 1000

# # Initialize wandb for tracking
# wandb.init(
#     project="Gaps-DPR", 
#     entity="ayoungren-colostate",
#     name=f"DPR_lr-{learning_rate}_bs-{batch_size}_epochs-{epochs}",
#     config={
#         "learning_rate": learning_rate,
#         "batch_size": batch_size,
#         "epochs": epochs,
#         "neg_sample_size": neg_sample_size,
#         "warmup_steps": warmup_steps
#     }
# )

# optimizer = torch.optim.AdamW(list(question_encoder.parameters()) + list(context_encoder.parameters()), lr=1e-5)

# epochs = 3
# for epoch in range(epochs):
#     question_encoder.train()
#     context_encoder.train()
#     total_loss = 0
#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
#         # Move data to device (GPU/CPU)
#         question_input_ids = batch['question_input_ids'].to(device)
#         context_input_ids = batch['context_input_ids'].to(device)
        
#         # Forward pass
#         question_outputs = question_encoder(input_ids=question_input_ids)
#         context_outputs = context_encoder(input_ids=context_input_ids)
        
#         # Get embeddings (use the [CLS] token embeddings for simplicity)
#         question_embeddings = question_outputs.last_hidden_state[:, 0, :]
#         context_embeddings = context_outputs.last_hidden_state[:, 0, :]
        
#         # Compute loss
#         loss = compute_loss(question_embeddings, context_embeddings)
#         total_loss += loss.item()

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch + 1} - Loss: {total_loss / len(train_dataloader)}")

# # 5. Save the model
# torch.save(question_encoder.state_dict(), "question_encoder.pth")
# torch.save(context_encoder.state_dict(), "context_encoder.pth")